# Introduction to Quantum Machine Learning (QML)
## Installation
We will mostly use qml (http://www.qmlcode.org/) to generate the molecular representations and use some custom atomic/local kernels for kernel ridge regression (KRR).

First you install qml (I recommend the develop branch which is more up to date, and is needed for e.g. the FCHL19 representation) as detailed here http://www.qmlcode.org/installation.html. Just check the install works:

In [1]:
import qml

## Theory
In QML we always use KRR to regress a (quantum) property $y$ for a query molecule represented by a vector $\mathbf{X'}$. We expand the property of a query molecule in a basis of kernel functions placed on N training molecules represented by vectors $\mathbf{X}$: 

$$
y(\mathbf{X'}) = \sum_i^N \alpha_i K(\mathbf{X'}, \mathbf{X}_i)
$$


In matrix form this reads: 

$$
\mathbf{y} = \mathbf{K} \mathbf{\alpha}
$$


where $\mathbf{y}$ is a vector containing predicted properties, $\mathbf{K}$ is the kernel matrix containing the pairwise kernel elements between molecules in the training and prediction set and $\mathbf{\alpha}$ is the vector containing the regression coefficients, which can be obtained by minimizing the cost function:

$$
J(\mathbf{\alpha} | \{\mathbf{y}^{\mathrm{ref}}\}) = \frac{1}{2} ||\mathbf{K} \alpha - \mathbf{y}^{\mathrm{ref}} ||^2_2 + \frac{\lambda}{2} \alpha^T \mathbf{K} \alpha
$$


where $\lambda$ is the regularization coefficient. The minimum of the cost function has the following closed-form solution:

$$
\alpha = (\mathbf{K} + \mathbf{I}\lambda)^{-1} \mathbf{y}^{\mathrm{ref}}
$$


In QML we distinguish between global and local kernels. Global kernels use a single vector to represent the molecule $\mathbf{X}$ and a corresponding global kernel function:

$$
K_{ij} = K(\mathbf{X}_i, \mathbf{X}_j) = \exp\Big(-\frac{||\mathbf{X}_i - \mathbf{X}_j||_2^2}{2 \sigma^2}\Big)
$$

which is here a gaussian function with width $\sigma$ with Euclidean norm but could be any other smooth and differentiable function. 


A local kernel on the other hand is to be used in combination with local representations for atoms in molecules $\mathbf{q}$:

$$
K_{ij} = \sum_{I\in i}\sum_{J\in j} \delta_{Z_I Z_J} \exp\Bigg(-\frac{||\mathbf{q}_I - \mathbf{q}_J||^2_2}{2\sigma^2} \Bigg)
$$

where here $\mathbf{q}_I$ and $\mathbf{q}_J$ are the representations of the Ith and Jth atoms in molecules i and j respectively. The Kronecker delta $\delta$ is not included in all representations but is for example in the kernel for FCHL. $Z_I$ and $Z_J$ are the atomic numbers (nuclear charges) of each atom, respectively.

## Dataset 
Here we will use the QM7 dataset, a dataset of atomization energies and relaxed geometries at PBE0/def2-TZVP level of theory for approx. 7k small organic molecules (up to 7 heavy atoms).

## Using QML
QML has `Compound` objects that read the nuclear charges and coordinates from xyz files:

In [2]:
mol = qml.Compound(xyz="qm7/0001.xyz")

In [3]:
mol.nuclear_charges

array([6, 1, 1, 1, 1])

In [4]:
mol.coordinates

array([[ 1.041682, -0.0562  , -0.071481],
       [ 2.130894, -0.056202, -0.071496],
       [ 0.678598,  0.174941, -1.072044],
       [ 0.678613,  0.694746,  0.62898 ],
       [ 0.678614, -1.038285,  0.228641]])

## Generating global representations with QML 
### Coulomb matrices
The Coulomb matrix $\mathbf{X}$ (Rupp et al., Phys. Rev. Lett., 108, 058301, 2012) is defined as: 

$$
M_{ij} = 
\begin{cases}
  0.5 Z_i^{2.4}  & \mathrm{for} \ i = j\\
  \frac{Z_i Z_j}{R_{ij}}  & \mathrm{for} \ i \neq j
\end{cases}
$$

where the diagonal elements are the interactions of atoms with themselves (effectively a polynomial fit of the atomic energies to the nuclear charge $Z_i$) and the off-diagonal elements represent the Coulomb repulsion between nuclei $i$ and $j$. 

Normally the Coulomb matrix rows and columns are then sorted by their L2-norm to handle permutation invariance.

We can generate the (sorted) coulomb matrix for a qml Compound object as follows:

In [5]:
from qml import representations

In [6]:
m = representations.generate_coulomb_matrix(
mol.nuclear_charges, mol.coordinates, size=len(mol.nuclear_charges),
sorting='row-norm')

In [7]:
m

array([36.8581052 ,  5.50857022,  0.5       ,  5.50857007,  0.56221501,
        0.5       ,  5.5085695 ,  0.56221605,  0.56221611,  0.5       ,
        5.50856526,  0.56221669,  0.56221777,  0.56221405,  0.5       ])

### Bag of Bonds 

The Bag of Bonds representation (Hansen et al., J. Phys. Chem. Lett., 2015, 6, 2326-2331) expands on the CM representation. For each element a bag (vector) is constructed for a self interaction and a pairwise interaction (e.g. C-C, C-O, C-N, etc). 

The self interaction is as in the CM:

$$
\frac{1}{2} Z_I^{2.4}
$$


The pairwise interacton between atom $i$ of element $I$ and atom $j$ of element $J$ is given by:

$$
\frac{Z_I Z_J}{|\mathbf{R}_i - \mathbf{R}_j|}
$$

where $\mathbf{R}_i$ is the euclidean coordinates of an atom $i$. 

The sorted bags are concatenated to a 1D vector representation.

In [8]:
import numpy as np

In [9]:
mol.nuclear_charges

array([6, 1, 1, 1, 1])

In [10]:
atomtypes = np.unique(mol.nuclear_charges)
atomtypes

array([1, 6])

In [11]:
size = len(mol.nuclear_charges)
size

5

In [12]:
bob = representations.generate_bob(mol.nuclear_charges,
                                  mol.coordinates,
                                  atomtypes,
                                  size=size,
                                  asize={
                                      "C":1,
                                      "O":0,
                                      "N":0,
                                      "H":4
                                  })

In [13]:
bob

array([36.8581052 ,  5.50857022,  5.50857007,  5.5085695 ,  5.50856526,
        0.5       ,  0.5       ,  0.5       ,  0.5       ,  0.56221777,
        0.56221669,  0.56221611,  0.56221605,  0.56221501,  0.56221405])

In [14]:
bob.shape

(15,)

## Training a KRR model from global features
For any global representation, taking as an example the CM, they can be used to train a KRR model like: 

In [15]:
from glob import glob

In [16]:
xyzs = sorted(glob("qm7/*.xyz"))

In [17]:
mols = [qml.Compound(x) for x in xyzs]

In [18]:
max_natoms = max([len(mol.nuclear_charges) for mol in mols])

In [19]:
reps = [representations.generate_coulomb_matrix(
mol.nuclear_charges, mol.coordinates, size=max_natoms,
sorting='row-norm') for mol in mols]

In [20]:
# read PBE0 (DFT) data

In [21]:
import pandas as pd

In [22]:
energies = pd.read_csv('hof_qm7.txt', header=None, sep='\s')
energies.columns = ['filename', 'PBE0', 'DFTB']

/Users/puckvg/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [23]:
energies['filename'].to_list() == xyzs

True

In [24]:
energies

,filename,PBE0,DFTB
0,qm7/0001.xyz,-417.031,-431.787272
1,qm7/0002.xyz,-711.117,-730.835895
2,qm7/0003.xyz,-563.084,-573.104249
3,qm7/0004.xyz,-403.695,-412.902289
4,qm7/0005.xyz,-858.499,-869.887692
...,...,...,...
7096,qm7/7168.xyz,-1224.850,-1339.264266
7097,qm7/7169.xyz,-1065.110,-1180.593348
7098,qm7/7170.xyz,-1309.130,-1446.772902
7099,qm7/7171.xyz,-1296.090,-1430.446486


In [25]:
y = energies.PBE0

In [26]:
from sklearn.model_selection import train_test_split

In [27]:
from sklearn.metrics.pairwise import rbf_kernel

In [28]:
X_train, X_test, y_train, y_test = train_test_split(reps, y)

In [29]:
# hyperparams to opt (fix for now)
sigma = 100
l2reg = 1e-8

In [30]:
gamma = 1. / (2 * sigma ** 2)

In [31]:
K = rbf_kernel(X_train, X_train, gamma=gamma)

In [32]:
K[np.diag_indices_from(K)] += l2reg

In [33]:
alpha = np.dot(np.linalg.inv(K), y_train)

In [34]:
K_test = rbf_kernel(X_test, X_train, gamma=gamma)

In [35]:
y_pred = np.dot(K_test, alpha)

In [36]:
mae = np.mean(np.abs(y_test - y_pred))

In [37]:
mae

8.33677468604611

## Generating local representations with QML

### aSLATM and SLATM

The aSLATM and SLATM representations (Huang and von Lilienfeld, Nat. Chem. 12, 945-951, 2020) is constructed by considering first the environments around single atoms (one body), then a potential resulting from pairs of atomic environments (two body) and another potential resulting from combinations of three atomic environments (three body).

The one-body term for an atom $I$ is just the nuclear charge $Z_I$. 

The two-body term is given by: 

$$
\frac{1}{2}\sum_{J \neq I} Z_J \delta(\mathbf{r} - R_{IJ})g(\mathbf{r})
$$

where $g(r)$ is the London potential $g(r) = 1/r^6$.


And the three-body term is given by: 

$$
\frac{1}{3} \sum_{J \neq K \neq I} Z_J Z_K \delta(\theta - \theta_{IJK}) h(\theta, \mathbf{R}_{IJ}, \mathbf{R}_{IK})
$$

where $\theta$ is the angle spanned by the vector $\mathbf{R}_{IJ}$ and $\mathbf{R}_{IK}$ (i.e. $\theta_{IJK}$). The three-body contribution $h(\theta, \mathbf{R}_{IJ}, \mathbf{R}_{IK})$ is based on the Axilrod-Teller-Muto (ATM) potential:

$$
h(\theta, \mathbf{R}_{IJ}, \mathbf{R}_{JK} = \frac{1 + \cos\theta \cos\theta_{JKI} \cos\theta_{KIJ}}{(\mathbf{R}_{IJ} \mathbf{R}_{IK} \mathbf{R}_{KJ})^3}
$$

where:

$$
\theta_{JKI} = f_1(\theta, \mathbf{R}_{IJ}, \mathbf{R}_{IK}), \theta_{KIJ} = f_2(\theta, \mathbf{R}_{IJ}, \mathbf{R}_{IK}), \mathbf{R}_{KJ} = f_3(\theta, \mathbf{R}_{IJ}, \mathbf{R}_{IK})
$$


The one, two and three body terms are concatenated for each atomtype to get the aSLATM representation. 

A global version, SLATM, can be obtained by summing over all atomic contributions.

Both can be generated using the `qml` package, where the combinations of nuclear charges (atomtypes) to be considered (in a one, two, three body way) is first specified before constructing the representation:

In [38]:
mbtypes = qml.representations.get_slatm_mbtypes([mol.nuclear_charges])

In [39]:
aslatm = np.array(qml.representations.generate_slatm(mol.coordinates, 
                                          mol.nuclear_charges,
                                          mbtypes,
                                          local=True))

In [40]:
aslatm.shape

(5, 857)

You can see there is one representation per nuclear charge. This local representation has a corresponding local kernel which also has an argument for the number of atoms $N$:

Note we cut the training set size for efficiency.

In [41]:
reps = np.array([np.array(qml.representations.generate_slatm(mol.coordinates, 
                                          mol.nuclear_charges,
                                          mbtypes,
                                          local=True))
       for mol in mols[:500]])

/Users/puckvg/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """


In [42]:
y = y[:500]

In [43]:
natoms = np.array([len(mol.nuclear_charges) for mol in mols[:500]])

In [44]:
X_train, X_test, N_train, N_test, y_train, y_test = train_test_split(
reps, natoms, y)

In [45]:
X_train = np.concatenate(X_train)
X_test = np.concatenate(X_test)

In [46]:
K = qml.kernels.get_local_kernels_gaussian(X_train, X_train,
                                          N_train, N_train,
                                          [sigma])[0]

In [47]:
K[np.diag_indices_from(K)] += l2reg

In [48]:
alpha = np.dot(np.linalg.inv(K), y_train)

In [49]:
K_test = qml.kernels.get_local_kernels_gaussian(X_test, X_train, 
                                                N_test, N_train,
                                                [sigma])[0]

In [50]:
y_pred = np.dot(K_test, alpha)

In [51]:
mae = np.mean(np.abs(y_test - y_pred))

In [52]:
mae

3.212686569335936

The global version can be generated using the `local=False` flag:

In [53]:
slatm = np.array(qml.representations.generate_slatm(mol.coordinates,
                                                   mol.nuclear_charges,
                                                   mbtypes,
                                                   local=False))

In [54]:
slatm.shape

(857,)

Which can be used with a regular RBF kernel like the global CM.

## FCHL

There are two variants of the FCHL representation: the original, dubbed FCHL18 (Faber et al, J. Chem. Phys., 148, 241717, 2018), and a newer variant dubbed FCHL19 (Christensen et al, J. Chem. Phys., 152, 044107, 2020). 

### FCHL18
FCHL18 is based on Gaussian distribution functions scaled by power laws which explicitly counts for structural and elemental degrees of freedom. Like SLATM, there are first, second and third order terms. The first order expansion $A_1(I)$ accounts for chemical composition and is modelled by a Gaussian function placed on period $P_I$ and group $G_I$ in the periodic table of element $I$:

$$
A_1(I) = \mathcal{N}(\mathbf{x}_I^{(1)}) = \exp\Big(-\frac{(P_I - \chi_1)^2}{2\sigma_P^2} - \frac{(G_I - \chi_2)^2}{2\sigma_G^2} \Big)
$$

where $\mathbf{x}_I^{(1)} = \{P_I, \sigma_P; G_I, \sigma_G\}$ with respective widths $\sigma_P$ and $\sigma_G$, which can be seen as elemental smearing parameters which control the nearsightedness of elements in the periodic table. $\chi_1$ and $\chi_2$ represent dummy variables for period and group which are integrated out when evaluating the Euclidean distance. 

The second order expansion $A_2(I)$ is a product of $A_1(I)$ and a sum that runs over neighbouring atoms $i$: 

$$
A_2(I) = \mathcal{N}(\mathbf{x}_I^{(1)}) \sum_{i \neq I} \mathcal{N}(\mathbf{x}_{iI}^{(2)}) \xi_2(d_{iI})
$$

where: 

$$
\mathbf{x}_{iI}^{(2)} = \{d_{iI}, \sigma_d; P_i, \sigma_P; G_i, \sigma_G\}
$$

where $d_{iI}$ and $\sigma_d$ correspond to the interatomic distance where a Gaussian is placed and its width. $\xi_2$ corresponds to the 2-body scaling function which takes the form of a power law.


Finally the third order expansion $A_3(I)$ runs over all neighbouring atoms: 

$$
A_3(I) = \mathcal{N}(\mathbf{x}^{(1)}) \sum_{i \neq I} \mathcal{N}(\mathbf{x}^{(2)}_{iI}) \sum_{j \neq i, I} \mathcal{N}(\mathbf{x}_{ijI}^{(3)}) \xi_3(d_{iI}, d_{jI}, \theta_{ij}^I)
$$

where: 

$$
\mathbf{x}_{ijI}^{(3)} = \{\theta_{ij}^I, \sigma_{\theta}; P_j, \sigma_P; G_j, \sigma_G \}
$$

where $P_j$ and $G_j$ correspond to the period and group of atom $j$. $\xi_3$ is the three-body scaling function, $\theta_{ij}^I$ is the principle angle between the two distance vectors which span from $I$ to $i$ and $I$ to $j$ respectively. $\sigma_{\theta}$ is the width of the Gaussian placed at $\theta_{ij}^I$.

For the forms of the scaling functions please refer to the paper. 
FCHL18 is constructed to be used with its own kernel with specific dimensions:

In [55]:
rep = np.array(qml.fchl.generate_representation(
mol.coordinates, mol.nuclear_charges, max_size=max_natoms))

In [56]:
rep.shape

(23, 5, 23)

As for the Coulomb matrix example above, a `max_size` flag is used to pad representations to a maximum dimension.

The FCHL18 representation is 3D and has a specific kernel to handle these dimensions:

In [57]:
mols = [qml.Compound(x) for x in xyzs]

In [58]:
max_natoms = max([len(mol.nuclear_charges) for mol in mols])

In [59]:
reps = np.array([qml.fchl.generate_representation(
mol.coordinates, mol.nuclear_charges, max_size=max_natoms,
) for mol in mols[:500]])

In [60]:
X_train, X_test, y_train, y_test = train_test_split(reps, y)

In [61]:
# hyperparams to opt (fix for now)
sigma = 5
l2reg = 1e-8

In [62]:
K = qml.fchl.get_local_symmetric_kernels(X_train, sigma)[0]

In [63]:
K[np.diag_indices_from(K)] += l2reg

In [64]:
alpha = np.dot(np.linalg.inv(K), y_train)

In [65]:
K_test = qml.fchl.get_local_kernels(X_test, X_train, sigma)[0]

In [66]:
K.shape

(375, 375)

In [67]:
y_pred = np.dot(K_test, alpha)

In [68]:
mae = np.mean(np.abs(y_test - y_pred))

In [69]:
mae

0.8068252896528206

### FCHL19
Whereas FCHL18 solves an analytical integral to compare atomic environments, FCHL19 uses a discretised representation to enable greater computational efficiency. The representation is a vector per atomic environment in an atom, like aSLATM.

The two body term encodes radial distributions between the central atoms and neighbouring atoms of a given element type. The three body term encodes the mean distances and angles between an atom and neighbouring pairs of atoms of given element types. It does not include an explicit one body term.

The two body term has the form:

$$
G^{\mathrm{2-body}} = \xi_2(r_{IJ})f_{\mathrm{cut}}(r_{IJ}) \exp\Bigg(-\frac{(\ln R_s - \mu(r_{ij}))^2}{2\sigma(r_{ij})^2}\Bigg)
$$

where $R_s$ is the distance location of the grid point and $\mu(r_{ij})$ and $\sigma(r_{ij})$ are parameters of the log-normal distribution which in turn depend on the interatomic distance $r_{IJ}$ and a hyperparameter $w$:

$$
\mu(r_{ij}) = \ln \Big( \frac{r_{IJ}}{\sqrt{1 + w/r^2_{IJ}}} \Big)
$$

and 

$$
\sigma(r_{ij})^2 = \ln\Bigg(1 + \frac{w}{r_{IJ}^2} \Bigg)
$$

with the two body scaling function $\xi_2(r_{IJ})$: 

$$
\xi_2(r_{IJ}) = \frac{1}{r_{IJ}^{N_2}}
$$

where the exponent $N_2$ is a hyperparameter of the representation. A soft cut-off function is used.

In [70]:
elements = np.unique(np.concatenate([(mol.nuclear_charges) for mol in mols]))

In [71]:
rep = qml.representations.generate_fchl_acsf(mol.nuclear_charges, 
                                            mol.coordinates,
                                            elements=elements,
                                            gradients=False,
                                            pad=max_natoms)

In [72]:
rep.shape

(23, 720)

The representation is 2D like aSLATM but it is recommended to use a kernel which includes a Dirac delta function over nuclear charges, implemented in `qml`. For this kernel the nuclear charges of the train and test molecules need to be provided.

In [73]:
reps = np.array([qml.representations.generate_fchl_acsf(mol.nuclear_charges, 
                                            mol.coordinates,
                                            elements=elements,
                                            gradients=False,
                                            pad=max_natoms)
        for mol in mols[:500]])

In [74]:
nuclear_charges = np.array([mol.nuclear_charges for mol in mols[:500]])

/Users/puckvg/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """Entry point for launching an IPython kernel.


In [75]:
X_train, X_test, Q_train, Q_test, y_train, y_test = train_test_split(
    reps, nuclear_charges, y)

In [76]:
K = qml.kernels.get_local_symmetric_kernel(X_train, Q_train, sigma)

In [77]:
K[np.diag_indices_from(K)] += l2reg

In [78]:
alpha = np.dot(np.linalg.inv(K), y_train)

In [79]:
K_test = qml.kernels.get_local_kernel(X_train, X_test, Q_train, Q_test,
                                     sigma)

In [80]:
y_pred = np.dot(K_test, alpha)

In [81]:
mae = np.mean(np.abs(y_test - y_pred))

In [82]:
mae

1.0355328826656074

### Other local representations

There are other popular local representations, perhaps the most notable being SOAP and ACSF. SOAP in particular is not very easy to understand to non physicists / quantum chemists so I won't mention it here. The most important thing to understand about these representations is:
- they are constructed directly from nuclear charges and coordinates read in from xyz files 
- they are constructed in order to ensure certain properties: uniqueness, invariance to changes in atom indexing,  translation, rotation and atom reflection
- this is because the property we want to learn (i.e. an energy) will not change if we re-order the atom indexing, translate the molecules, rotate them, or reflect them in a mirror plane 

For a nice review on this, please refer to this review: 
https://arxiv.org/pdf/2003.12081.pdf

Others are now trying to incorporate these symmetries / physical laws into neural networks to predict quantum properties.